In [1]:
import requests
import re
from bs4 import BeautifulSoup
from kafka import KafkaProducer
import json

In [2]:
url='https://öffi.at/?archive=1&text=&types=2%2C3&page='
kafka_server = 'localhost:29092'

In [3]:
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')
pages=int(soup.find(string=re.compile(r'Aktuelle Seite: \d+/\d+')).split('/')[-1].strip()[:-1])

In [4]:
producer = KafkaProducer(bootstrap_servers=kafka_server,value_serializer=lambda v: json.dumps(v).encode('utf-8'))

In [5]:
#for n in range(1,pages+1):
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')
toplist=soup.find('ul',{'class':'category-filter'})
for delay in toplist.findChildren('li',attrs={'class': 'disruption uk-padding-small'},recursive=False):
    lines=[]
    stations=[]
    result={}
    id=delay.attrs['id']
    title=delay.find('h2',{'class':'disruption-title'}).text.split(':')[-1].strip()
    content=delay.find('div',{'class':'uk-accordion-content'})
    for line in content.find_all('ul')[0].find_all('li'):
        lines.append(line.text)
    for station in content.find_all('ul')[1].find_all('li'):
        stations.append(station.text)
    start=content.find_all(string=re.compile(r': \d{2}\.\d{2}\.\d{4} \d{2}:\d{2}'))[0].split(': ')[1]
    end=content.find_all(string=re.compile(r': \d{2}\.\d{2}\.\d{4} \d{2}:\d{2}'))[1].split(': ')[1]
    result={'id':id,'title':title,'lines':lines,'stations':stations,'start':start,'end':end}
    producer.send('delays', result)
    producer.flush()